<a href="https://colab.research.google.com/github/ridamjain09/NLP/blob/main/LSTM_Biirectional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('spam_utf8.csv')

In [8]:
df.head()

,v1,v2,labels
0,ham,"Go until jurong point, crazy.. Available only ...",1
1,ham,Ok lar... Joking wif u oni...,1
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0
3,ham,U dun say so early hor... U c already then say...,1
4,ham,"Nah I don't think he goes to usf, he lives aro...",1


In [6]:
df = df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis = 1)

In [7]:
df['labels'] = df['v1'].map({'spam':0 , 'ham': 1})

In [11]:
#renaming the coloum v2
df.rename(columns={'v2': 'reviews'}, inplace=True)

In [15]:
df.drop(['v1'], axis = 1, inplace = True)

In [16]:
df.head()

,reviews,labels
0,"Go until jurong point, crazy.. Available only ...",1
1,Ok lar... Joking wif u oni...,1
2,Free entry in 2 a wkly comp to win FA Cup fina...,0
3,U dun say so early hor... U c already then say...,1
4,"Nah I don't think he goes to usf, he lives aro...",1


In [17]:
X = df['reviews']
y = df['labels']

In [18]:
import tensorflow as tf


In [19]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [20]:

messages = df.copy()

In [21]:
messages.head()

,reviews,labels
0,"Go until jurong point, crazy.. Available only ...",1
1,Ok lar... Joking wif u oni...,1
2,Free entry in 2 a wkly comp to win FA Cup fina...,0
3,U dun say so early hor... U c already then say...,1
4,"Nah I don't think he goes to usf, he lives aro...",1


In [ ]:
#preprocessing the dataset

from nltk.stem.porter import PorterStemmer
#Lemmetization

ps = PorterStemmer()

In [24]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(0,len(messages)):
  reply = re.sub('^[a-zA-Z0-9]', ' ',messages['reviews'][i])
  reply = reply.lower()
  reply = reply.split()
  reply = [lemmatizer.lemmatize(word) for word in reply if not word in stopwords.words('english')]
  reply = ' '.join(reply)
  corpus.append(reply)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
corpus

['jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...',
 'k lar... joking wif u oni...',
 "ree entry 2 wkly comp win fa cup final tkts 21st may 2005. text fa 87121 receive entry question(std txt rate)t&c's apply 08452810075over18's",
 'dun say early hor... u c already say...',
 'ah think go usf, life around though',
 "reemsg hey darling 3 week's word back! i'd like fun still? tb ok! xxx std chgs send, å£1.50 rcv",
 'ven brother like speak me. treat like aid patent.',
 "per request 'melle melle (oru minnaminunginte nurungu vettam)' set callertune callers. press *9 copy friend callertune",
 'inner!! valued network customer selected receivea å£900 prize reward! claim call 09061701461. claim code kl341. valid 12 hour only.',
 'ad mobile 11 month more? u r entitled update latest colour mobile camera free! call mobile update co free 08002986030',
 "'m gonna home soon want talk stuff anymore tonight, k? i've cried enough today.",
 'ix chance win cash! 100 

In [26]:
#Using OHE
voc_size = 5000
onehot_repr = [one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2055,
  4668,
  331,
  1313,
  3199,
  437,
  810,
  251,
  227,
  1240,
  1007,
  3202,
  3620,
  2680,
  3222],
 [1937, 1987, 2834, 4046, 721, 2123],
 [2878,
  1047,
  1837,
  3066,
  3331,
  2453,
  1508,
  520,
  4223,
  4250,
  2806,
  1873,
  2221,
  810,
  1508,
  4958,
  2850,
  1047,
  3764,
  2328,
  2676,
  45,
  426,
  1898,
  3631,
  2544],
 [296, 279, 3490, 2112, 721, 1156, 282, 279],
 [4246, 4377, 3863, 625, 217, 904, 2016],
 [3437,
  859,
  2810,
  1468,
  3473,
  4135,
  3682,
  3204,
  139,
  4179,
  2200,
  2862,
  4209,
  944,
  2328,
  2489,
  4945,
  4271,
  1463,
  3277],
 [2140, 3873, 139, 482, 2137, 485, 139, 4573, 2318],
 [2039,
  4137,
  795,
  4344,
  3770,
  1328,
  1348,
  729,
  235,
  1441,
  3618,
  1037,
  1827,
  2073,
  607,
  4690,
  3618],
 [2934,
  219,
  4923,
  1109,
  1529,
  2541,
  4458,
  1281,
  4717,
  4191,
  3013,
  345,
  4191,
  1812,
  4139,
  3540,
  2977,
  274,
  4235],
 [4392,
  2071,
  3188,
  2333,
  2333,
  721,
  3954,
  152

In [37]:
#Padding
sent_len = 20
embedded_doc  = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_len)

In [36]:
#Finding max length for a sentence
max_lenght = max((df['reviews'].apply(len)))
max_lenght

910

In [50]:
#Creating a model
embedding_features = 35
model = Sequential()
model.add(Embedding(voc_size,embedding_features,input_length = sent_len))
model.add(Bidirectional(LSTM(850)))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer ='adam',metrics =['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 20, 35)            175000    
                                                                 
 bidirectional_5 (Bidirecti  (None, 1700)              6024800   
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 1)                 1701      
                                                                 
Total params: 6201501 (23.66 MB)
Trainable params: 6201501 (23.66 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [44]:
X.shape

(5572,)

In [45]:
import numpy as np
x_final = np.array(embedded_doc)
y_final = np.array(y)

In [46]:
#Train Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size = 0.2, random_state = 42)

In [51]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 10 ,batch_size = 30)

Epoch 1/10
149/149 [==============================] - 141s 920ms/step - loss: 0.1413 - accuracy: 0.9569 - val_loss: 0.0859 - val_accuracy: 0.9803
Epoch 2/10
149/149 [==============================] - 130s 873ms/step - loss: 0.0405 - accuracy: 0.9881 - val_loss: 0.0826 - val_accuracy: 0.9749
Epoch 3/10
149/149 [==============================] - 131s 879ms/step - loss: 0.0786 - accuracy: 0.9834 - val_loss: 0.1119 - val_accuracy: 0.9632
Epoch 4/10
149/149 [==============================] - 129s 869ms/step - loss: 0.0230 - accuracy: 0.9951 - val_loss: 0.1390 - val_accuracy: 0.9803
Epoch 5/10
149/149 [==============================] - 129s 869ms/step - loss: 0.0086 - accuracy: 0.9982 - val_loss: 0.1205 - val_accuracy: 0.9821
Epoch 6/10
149/149 [==============================] - 131s 880ms/step - loss: 0.0023 - accuracy: 0.9993 - val_loss: 0.1222 - val_accuracy: 0.9812
Epoch 7/10
149/149 [==============================] - 131s 877ms/step - loss: 3.1219e-04 - accuracy: 1.0000 - val_loss: 0.15

In [52]:
y_pred = model.predict(X_test)


35/35 [==============================] - 11s 241ms/step


In [54]:
y_pred = np.where(y_pred > 0.5,1,0)

In [56]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))


[[131  19]
 [  3 962]]
0.9802690582959641
              precision    recall  f1-score   support

           0       0.98      0.87      0.92       150
           1       0.98      1.00      0.99       965

    accuracy                           0.98      1115
   macro avg       0.98      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115

